# MultinomialNaive Bayes using SKLearn Library

In [1]:
# Imports

import pandas as pd
import numpy as np
import nltk
import re

In [2]:
import requests

In [3]:
from sklearn.naive_bayes import MultinomialNB

In [4]:
cleaned_single_annotator = pd.read_csv("training_clean/clean_single_annotator.csv",delimiter=",")
cleaned_double_annotator_agree = pd.read_csv("training_clean/clean_double_annotator_agree.csv",delimiter=",")
cleaned_double_annotator_disagree = pd.read_csv("training_clean/clean_double_annotator_disagree.csv",delimiter=",")
cleaned_triple_annotator_agree = pd.read_csv("training_clean/clean_triple_annotator_agree.csv",delimiter=",")
cleaned_triple_annotator_disagree = pd.read_csv("training_clean/clean_triple_annotator_disagree.csv",delimiter=",")

In [5]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq
0,0,1001238,mengejar,2801,berencana pulang amerika serikat volume 42 ber...,1
1,1,1001238,mengejar,2802,berencana pulang amerika serikat volume 42 ber...,1
2,2,1001448,mengejar,2801,mekanisme memicu produksi nasional ketidakpuas...,1
3,3,1001448,mengejar,2802,mekanisme memicu produksi nasional ketidakpuas...,1
4,4,1001820,mengejar,2801,ganti kaum melayu melepaskan ketuanan melayuny...,1


In [6]:
# returns = requests.post('http://bahasa.cs.ui.ac.id/postag/API/tag',
#                          data={'Text[value]':'gambar dihasilkan layarnya cerah memiliki speaker menghasilkan suara keras jernih'})

## Tokenize sentences

In [7]:
cleaned_double_annotator_disagree['kalimat'] = cleaned_double_annotator_disagree['kalimat'].apply(nltk.word_tokenize)

In [8]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq
0,0,1001238,mengejar,2801,"[berencana, pulang, amerika, serikat, volume, ...",1
1,1,1001238,mengejar,2802,"[berencana, pulang, amerika, serikat, volume, ...",1
2,2,1001448,mengejar,2801,"[mekanisme, memicu, produksi, nasional, ketida...",1
3,3,1001448,mengejar,2802,"[mekanisme, memicu, produksi, nasional, ketida...",1
4,4,1001820,mengejar,2801,"[ganti, kaum, melayu, melepaskan, ketuanan, me...",1


In [10]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq
0,0,1001238,mengejar,2801,"[berencana, pulang, amerika, serikat, volume, ...",1
1,1,1001238,mengejar,2802,"[berencana, pulang, amerika, serikat, volume, ...",1
2,2,1001448,mengejar,2801,"[mekanisme, memicu, produksi, nasional, ketida...",1
3,3,1001448,mengejar,2802,"[mekanisme, memicu, produksi, nasional, ketida...",1
4,4,1001820,mengejar,2801,"[ganti, kaum, melayu, melepaskan, ketuanan, me...",1


## Trying to pick collocation range of 2

In [51]:
collocs = []
for row in cleaned_double_annotator_disagree.values:
    kata = row[2]
    kalimat = row[4]
    try:
        target_index = kalimat.index(kata)
    except ValueError:
        colloc = ['','','','']
        collocs.append(colloc)
        continue
    panjang_kalimat = len(kalimat)
    if target_index == 1:
        if panjang_kalimat - target_index > 2:
            colloc = ['',kalimat[target_index-1],kalimat[target_index+1],kalimat[target_index+2]]
        elif panjang_kalimat - target_index == 2:
            colloc = ['',kalimat[target_index-1],kalimat[target_index+1],'']
        else:
            colloc = ['',kalimat[target_index-1],'','']
    elif target_index == 0:
        if panjang_kalimat - target_index > 2:
            colloc = ['','',kalimat[target_index+1],kalimat[target_index+2]]
        elif panjang_kalimat - target_index == 2:
            colloc = ['','',kalimat[target_index+1],'']
    else:
        if panjang_kalimat - target_index > 2:
            colloc = [kalimat[target_index-2],kalimat[target_index-1],kalimat[target_index+1],kalimat[target_index+2]]
        elif panjang_kalimat - target_index == 2:
            colloc = [kalimat[target_index-2],kalimat[target_index-1],kalimat[target_index+1],'']
        else:
            colloc = [kalimat[target_index-2],kalimat[target_index-1],'','']
    collocs.append(colloc)
# collocs = np.array(collocs)

In [52]:
cleaned_double_annotator_disagree['colloc'] = collocs

In [59]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq,colloc,features
0,0,1001238,mengejar,2801,"[berencana, pulang, amerika, serikat, volume, ...",1,"[vermouth, berjanji, sherry, ]","[vermouth, berjanji, sherry, , berencana, pula..."
1,1,1001238,mengejar,2802,"[berencana, pulang, amerika, serikat, volume, ...",1,"[vermouth, berjanji, sherry, ]","[vermouth, berjanji, sherry, , berencana, pula..."
2,2,1001448,mengejar,2801,"[mekanisme, memicu, produksi, nasional, ketida...",1,"[muncul, film, kewajiban, mutunya]","[muncul, film, kewajiban, mutunya, mekanisme, ..."
3,3,1001448,mengejar,2802,"[mekanisme, memicu, produksi, nasional, ketida...",1,"[muncul, film, kewajiban, mutunya]","[muncul, film, kewajiban, mutunya, mekanisme, ..."
4,4,1001820,mengejar,2801,"[ganti, kaum, melayu, melepaskan, ketuanan, me...",1,"[melayu, dibantu, ketertinggalan, ekonominya]","[melayu, dibantu, ketertinggalan, ekonominya, ..."


## Creating Feature Vector to be fed to classifier

The features vector construction is:

[Word, Colloc1, Colloc2, Colloc3, Colloc4, sentence1, ... , sentencen]

In [78]:
features = []
for row in cleaned_double_annotator_disagree.values:
    cur = [row[2]]
    cur += row[6] + row[4]
    features.append(cur)
cleaned_double_annotator_disagree['features'] = features
# features = np.array(features)

In [82]:
cleaned_double_annotator_disagree.head()

,Unnamed: 0,kalimat_id,kata,sense,kalimat,freq,colloc,features
0,0,1001238,mengejar,2801,"[berencana, pulang, amerika, serikat, volume, ...",1,"[vermouth, berjanji, sherry, ]","[mengejar, vermouth, berjanji, sherry, , beren..."
1,1,1001238,mengejar,2802,"[berencana, pulang, amerika, serikat, volume, ...",1,"[vermouth, berjanji, sherry, ]","[mengejar, vermouth, berjanji, sherry, , beren..."
2,2,1001448,mengejar,2801,"[mekanisme, memicu, produksi, nasional, ketida...",1,"[muncul, film, kewajiban, mutunya]","[mengejar, muncul, film, kewajiban, mutunya, m..."
3,3,1001448,mengejar,2802,"[mekanisme, memicu, produksi, nasional, ketida...",1,"[muncul, film, kewajiban, mutunya]","[mengejar, muncul, film, kewajiban, mutunya, m..."
4,4,1001820,mengejar,2801,"[ganti, kaum, melayu, melepaskan, ketuanan, me...",1,"[melayu, dibantu, ketertinggalan, ekonominya]","[mengejar, melayu, dibantu, ketertinggalan, ek..."


## Trying Classifier

In [15]:
skl_clf = MultinomialNB()

### Failure happened below because SKLearn MNB could not take an imperfect 2D array (that is, a 2D array in which the element has different array length)

In [84]:
X = cleaned_double_annotator_disagree['features'].values
y = cleaned_double_annotator_disagree['sense'].values
skl_clf.fit(X, y)

ValueError: setting an array element with a sequence.

### So we will be using a custom MNB Classifier created as a result from Machine Learning Course Assignment

In [86]:
class MNB_TextClassifier:
    
    def __init__(self):
        self.prior = {}
        self.condprob = {}
    
    # p(class=y | {term0=x[0], term1=x[1], ...} )
    # please note that you dont add new data to vocabulary when predicting
    def proba_y_given_x(self, y, x):
        length = len(self.condprob)
        prob_y = self.prior[y]
        prob_x = 1
        for i in x:
            if(i in self.condprob.keys()):
                prob_x *= (self.condprob[i][y])
            else:
                prob_x *= ((1)/(self.dict_y_term[y] + length))
        return prob_y * prob_x
 
        #raise NotImplementedError()
    
    # p(c)
    def proba_y(self, y):
        return self.prior[y]

    # update self.prior[class] as p(class=class) 
    # update self.condprob[term][class] as p(term=term | class=class)
    def fit(self, X, y):
        dict_y = {i:list(y).count(i) for i in y}
        total_y = 0

        for i in dict_y:
            total_y += dict_y[i]
        for i in dict_y:
            self.prior[i] = dict_y[i]/total_y
        
        X_Plain = list()
        
        for i in X:
            for j in i:
                X_Plain.append(j)

        dict_x = {i:X_Plain.count(i) for i in X_Plain}
        dict_x_dict_y = {i:dict() for i in dict_x}
        dict_x_cond_y = {i:dict() for i in dict_x}

        for i in range(len(X)):
            for j in X[i]:
                if(i in dict_x_cond_y[j]):
                    dict_x_cond_y[j][i] += 1
                else:
                    dict_x_cond_y[j][i] = 1
        self.dict_y_term = {i:0 for i in y}
        for i in range(len(y)):
            self.dict_y_term[y[i]] += (len(X[i]))

        dict_y_last = {i:{j:0 for j in self.dict_y_term} for i in dict_x_cond_y}

        for i in dict_x_cond_y:
            for j in range(len(y)):
                if(j in dict_x_cond_y[i]):
                    dict_y_last[i][y[j]] += dict_x_cond_y[i][j] 
        self.condprob = {i:{j:0 for j in self.dict_y_term} for i in dict_x_cond_y}
        for i in dict_y_last:
            for j in dict_y_last[i]:

                self.condprob[i][j] = (dict_y_last[i][j] + 1)/(self.dict_y_term[j] + len(dict_x))
        #print(dict_x)
        #print(self.dict_y_term)
        #print(dict_y_last)
        #raise NotImplementedError()
        
    def predict_single(self, x):
        classes = [i for i in self.prior.keys()]
        likelihood = [self.proba_y_given_x(i,x) for i in classes]
        return classes[likelihood.index(max(likelihood))]
        #raise NotImplementedError()
        
    def predict(self, X):
        return [self.predict_single(x) for x in X]

In [87]:
cust_clf = MNB_TextClassifier()
cust_clf.fit(X, y)